In [1]:
import numpy as np
import pandas as pd
from transformers import pipeline

import nltk
nltk.download('vader_lexicon')

import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
df = pd.read_csv("/content/preprocessing_data.csv")
df.head()

,title,stars,text
0,Pantai Tanjung Gelam,5,pantai menakjubkan pasirpasir putih gakood
1,Pantai Tanjung Gelam,5,pantai tanjungak gakelam merupakan salah satu ...
2,Pantai Tanjung Gelam,5,salah satu pantai favorit bila karimunjawa ber...
3,Pantai Tanjung Gelam,5,pantai dengakanpasir putih bersih panorama lau...
4,Pantai Tanjung Gelam,5,the iconic tanjungak gakelam coconut trees


In [5]:
df['text']

,text
0,pantai menakjubkan pasirpasir putih gakood
1,pantai tanjungak gakelam merupakan salah satu ...
2,salah satu pantai favorit bila karimunjawa ber...
3,pantai dengakanpasir putih bersih panorama lau...
4,the iconic tanjungak gakelam coconut trees
...,...
714,stunningak
715,nice place for sunrise and hangakout with friends
716,dependingak on when you come and wind directio...
717,destinasi sangakat bagakus pengakamatan sun ri...


In [6]:
df = df.dropna()

In [9]:
# Pakai GPU
classifier = pipeline("sentiment-analysis", model="w11wo/indonesian-roberta-base-sentiment-classifier", device=-1)

# Batch prediction
batch_size = 64
hasil_label = []

for i in range(0, len(df), batch_size):
    teks_batch = df['text'].iloc[i:i+batch_size].tolist()
    pred_batch = classifier(teks_batch)
    hasil_label.extend([x['label'] for x in pred_batch])

df['sentimen_bert'] = hasil_label


Device set to use cpu


In [11]:
# Inisialisasi VADER
sid = SentimentIntensityAnalyzer()

# Fungsi klasifikasi sentimen dengan VADER
def vader_sentiment(text):
    scores = sid.polarity_scores(text)
    compound = scores['compound']
    if compound >= 0.05:
        return 'positive'
    elif compound <= -0.05:
        return 'negative'
    else:
        return 'neutral'

# Terapkan ke kolom cleaned_text
df['sentimen_vader'] = df['text'].apply(vader_sentiment)

In [12]:
df

,title,stars,text,sentimen,sentimen_bert,sentimen_vader
0,Pantai Tanjung Gelam,5,pantai menakjubkan pasirpasir putih gakood,negative,negative,neutral
1,Pantai Tanjung Gelam,5,pantai tanjungak gakelam merupakan salah satu ...,positive,positive,neutral
2,Pantai Tanjung Gelam,5,salah satu pantai favorit bila karimunjawa ber...,positive,positive,neutral
3,Pantai Tanjung Gelam,5,pantai dengakanpasir putih bersih panorama lau...,positive,positive,neutral
4,Pantai Tanjung Gelam,5,the iconic tanjungak gakelam coconut trees,positive,positive,neutral
...,...,...,...,...,...,...
714,Pantai Legon Lele,4,stunningak,negative,negative,neutral
715,Pantai Legon Lele,5,nice place for sunrise and hangakout with friends,positive,positive,positive
716,Pantai Legon Lele,3,dependingak on when you come and wind directio...,positive,positive,negative
717,Pantai Legon Lele,5,destinasi sangakat bagakus pengakamatan sun ri...,positive,positive,neutral


In [13]:
df.to_csv('label_data.csv', index=False)